In [4]:
! pip install kagglehub

# load the raw dataset

In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("abinash6000/sle-gene-expression-profiles-7000")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/abinash6000/sle-gene-expression-profiles-7000/versions/1


In [6]:
path = ' /root/.cache/kagglehub/datasets/abinash6000/sle-gene-expression-profiles-7000/versions/1'

In [7]:
import os

path = "/root/.cache/kagglehub/datasets/abinash6000/sle-gene-expression-profiles-7000/versions/1"

for root, dirs, files in os.walk(path):
    level = root.replace(path, "").count(os.sep)
    indent = " " * (4 * level)
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = " " * (4 * (level + 1))
    for f in files:
        print(f"{sub_indent}{f}")

1/
    SLE_Aggregated/
        SLE_Aggregated_2.csv
        SLE_Aggregated_15.csv
        SLE_Aggregated_8.csv
        SLE_Aggregated_18.csv
        SLE_Aggregated_4.csv
        SLE_Aggregated_20.csv
        SLE_Aggregated_6.csv
        SLE_Aggregated_12.csv
        SLE_Aggregated_16.csv
        SLE_Aggregated_19.csv
        SLE_Aggregated_3.csv
        SLE_Aggregated_13.csv
        SLE_Aggregated_14.csv
        SLE_Aggregated_17.csv
        SLE_Aggregated_5.csv
        SLE_Aggregated_7.csv
        SLE_Aggregated_1.csv
        SLE_Aggregated_11.csv
        SLE_Aggregated_10.csv
        SLE_Aggregated_9.csv
    Control/
        GSM1228917_6631_Control_SLE_M1_HGU133P.csv
        GSM1228881_6633_Control_SLE_M3_HGU133P.csv
        GSM1228872_9399_Control_SLE_M10_SPR134_HGU133P.csv
        GSM1228870_9396_Control_SLE_M7_SPR134_HGU133P.csv
        GSM1228882_6634_Control_SLE_M4_HGU133P.csv
        GSM1228876_6632_Control_SLE_M2_HGU133P.csv
        GSM1228879_6629_Control_SLE_G4_HGU133P.csv
 

# arrange in folders

In [8]:
import os
import shutil

# Define paths
base_path = "/root/.cache/kagglehub/datasets/abinash6000/sle-gene-expression-profiles-7000/versions/1"
control_folder = os.path.join(base_path, "Control")
sle_folder = os.path.join(base_path, "SLE")

# Create directories if not exist
os.makedirs(control_folder, exist_ok=True)
os.makedirs(sle_folder, exist_ok=True)

# Loop through files and move them accordingly
for file_name in os.listdir(base_path):
    file_path = os.path.join(base_path, file_name)

    if os.path.isfile(file_path):  # Ensure it's a file
        if "Control" in file_name:
            shutil.move(file_path, os.path.join(control_folder, file_name))
        else:
            shutil.move(file_path, os.path.join(sle_folder, file_name))

print("Files have been organized into 'Control' and 'SLE' folders.")


Files have been organized into 'Control' and 'SLE' folders.


In [9]:
import os

# Define paths
base_path = "/root/.cache/kagglehub/datasets/abinash6000/sle-gene-expression-profiles-7000/versions/1"
control_folder = os.path.join(base_path, "Control")
sle_folder = os.path.join(base_path, "SLE")

# Count files in each folder
control_count = len([f for f in os.listdir(control_folder) if os.path.isfile(os.path.join(control_folder, f))])
sle_count = len([f for f in os.listdir(sle_folder) if os.path.isfile(os.path.join(sle_folder, f))])

# Print results
print(f"Number of files in 'Control' folder: {control_count}")
print(f"Number of files in 'SLE' folder: {sle_count}")


Number of files in 'Control' folder: 20
Number of files in 'SLE' folder: 61


# Undersampling SLE to handle biasness while handling Data Loss

In [10]:
import os
import pandas as pd
import random

def aggregate_sle_data(sle_folder, output_folder, group_size=3):
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # List all SLE (infected) files
    sle_files = [f for f in os.listdir(sle_folder) if f.endswith(".csv")]

    # Randomly exclude 1 file to make 60 (if needed)
    if len(sle_files) > 60:
        sle_files.remove(random.choice(sle_files))

    # Shuffle and group files into sets of `group_size`
    random.shuffle(sle_files)
    file_groups = [sle_files[i:i + group_size] for i in range(0, len(sle_files), group_size)]

    for idx, group in enumerate(file_groups):
        dfs = [pd.read_csv(os.path.join(sle_folder, file)) for file in group]

        # Merge on X, Y and compute mean
        df_merged = dfs[0].copy()
        for df in dfs[1:]:
            df_merged[["MEAN", "STDV", "NPIXELS"]] += df[["MEAN", "STDV", "NPIXELS"]]
        df_merged[["MEAN", "STDV", "NPIXELS"]] /= group_size  # Average

        # Save the aggregated file
        output_path = os.path.join(output_folder, f"SLE_Aggregated_{idx+1}.csv")
        df_merged.to_csv(output_path, index=False)

    print(f"Aggregated {len(file_groups)} files saved in {output_folder}")

# Define paths
sle_folder = "/root/.cache/kagglehub/datasets/abinash6000/sle-gene-expression-profiles-7000/versions/1/SLE"
output_folder = "/root/.cache/kagglehub/datasets/abinash6000/sle-gene-expression-profiles-7000/versions/1/SLE_Aggregated"

# Run the function
aggregate_sle_data(sle_folder, output_folder)


Aggregated 20 files saved in /root/.cache/kagglehub/datasets/abinash6000/sle-gene-expression-profiles-7000/versions/1/SLE_Aggregated


In [11]:
aggregated_sle="/root/.cache/kagglehub/datasets/abinash6000/sle-gene-expression-profiles-7000/versions/1/SLE_Aggregated"
  # Get a sample CSV file from the Control folder
sample_file = next((f for f in os.listdir(aggregated_sle) if f.endswith(".csv")), None)

if sample_file:
    sample_path = os.path.join(aggregated_sle, sample_file)
    df = pd.read_csv(sample_path, nrows=5)  # Read only the first 5 rows
    print(f"Head of {sample_file}:")
    print(df.head())
else:
    print("No CSV files found ")


Head of SLE_Aggregated_2.csv:
   X  Y          MEAN         STDV  NPIXELS
0  0  0    106.333333    15.366667     25.0
1  1  0  16974.666667  2283.200000     25.0
2  2  0    138.666667    30.866667     25.0
3  3  0  17039.333333  2651.733333     25.0
4  4  0     94.000000    14.033333     25.0


# UnderSampling 5000 rows from each file

In [12]:
def get_x_y_range(csv_file):
    df = pd.read_csv(csv_file)
    x_min, x_max = df['X'].min(), df['X'].max()
    y_min, y_max = df['Y'].min(), df['Y'].max()
    num_rows = len(df)
    print(f"Number of rows in dataset: {num_rows}")
    return x_min, x_max, y_min, y_max

In [13]:
sample_file = "/root/.cache/kagglehub/datasets/abinash6000/sle-gene-expression-profiles-7000/versions/1/SLE_Aggregated/SLE_Aggregated_1.csv"
x_min, x_max, y_min, y_max= get_x_y_range(sample_file)
print(f"X range: {x_min} to {x_max}")
print(f"Y range: {y_min} to {y_max}")


Number of rows in dataset: 1354896
X range: 0 to 1163
Y range: 0 to 1163


In [14]:
import os
import pandas as pd
import numpy as np

def sample_random_from_chunks(csv_file, output_file, num_samples=5000):
    """Samples data by dividing into chunks of ~270 rows and selecting one random row per chunk."""

    df = pd.read_csv(csv_file)
    total_rows = len(df)

    step = max(total_rows // num_samples, 1)  # Compute chunk size (~270)

    # Randomly select one row per chunk
    sampled_rows = [df.iloc[i + np.random.randint(0, min(step, total_rows - i))] for i in range(0, total_rows, step)]

    # Convert list to DataFrame and ensure exactly `num_samples` rows
    sampled_df = pd.DataFrame(sampled_rows).head(num_samples)

    # Save the sampled dataset
    sampled_df.to_csv(output_file, index=False)

    print(f"Sampled {len(sampled_df)} rows from {csv_file} → {output_file}")

# Paths
base_output_folder = "/root/.cache/kagglehub/datasets/abinash6000/sle-gene-expression-profiles-7000/versions/1/Final_Sampled"
control_folder = "/root/.cache/kagglehub/datasets/abinash6000/sle-gene-expression-profiles-7000/versions/1/Control"
sle_folder = "/root/.cache/kagglehub/datasets/abinash6000/sle-gene-expression-profiles-7000/versions/1/SLE_Aggregated"

# Subfolder paths
control_sampled_folder = os.path.join(base_output_folder, "control_sampled")
sle_sampled_folder = os.path.join(base_output_folder, "aggregated_sle_sampled")

# Create directories
os.makedirs(control_sampled_folder, exist_ok=True)
os.makedirs(sle_sampled_folder, exist_ok=True)

# Sample 5000 points for each file in Control and SLE_Aggregated
for folder, output_subfolder in [(control_folder, control_sampled_folder), (sle_folder, sle_sampled_folder)]:
    for file in os.listdir(folder):
        if file.endswith(".csv"):
            input_path = os.path.join(folder, file)
            output_path = os.path.join(output_subfolder, file)
            sample_random_from_chunks(input_path, output_path, num_samples=5000)


Sampled 5000 rows from /root/.cache/kagglehub/datasets/abinash6000/sle-gene-expression-profiles-7000/versions/1/Control/GSM1228917_6631_Control_SLE_M1_HGU133P.csv → /root/.cache/kagglehub/datasets/abinash6000/sle-gene-expression-profiles-7000/versions/1/Final_Sampled/control_sampled/GSM1228917_6631_Control_SLE_M1_HGU133P.csv
Sampled 5000 rows from /root/.cache/kagglehub/datasets/abinash6000/sle-gene-expression-profiles-7000/versions/1/Control/GSM1228881_6633_Control_SLE_M3_HGU133P.csv → /root/.cache/kagglehub/datasets/abinash6000/sle-gene-expression-profiles-7000/versions/1/Final_Sampled/control_sampled/GSM1228881_6633_Control_SLE_M3_HGU133P.csv
Sampled 5000 rows from /root/.cache/kagglehub/datasets/abinash6000/sle-gene-expression-profiles-7000/versions/1/Control/GSM1228872_9399_Control_SLE_M10_SPR134_HGU133P.csv → /root/.cache/kagglehub/datasets/abinash6000/sle-gene-expression-profiles-7000/versions/1/Final_Sampled/control_sampled/GSM1228872_9399_Control_SLE_M10_SPR134_HGU133P.csv
Sam

# download the final sampled data

In [15]:
import shutil

# Define path to final sampled folder
final_sampled_path = "/root/.cache/kagglehub/datasets/abinash6000/sle-gene-expression-profiles-7000/versions/1/Final_Sampled"
zip_path = "/root/Final_Sampled.zip"

# Create a zip archive
shutil.make_archive(zip_path.replace(".zip", ""), 'zip', final_sampled_path)

print(f"Zipped folder saved at: {zip_path}")


Zipped folder saved at: /root/Final_Sampled.zip


In [16]:
from google.colab import files

# Download the zip file
files.download(zip_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>